### OpenAI Fine-Tuning API
Detailed process of how I will be using OpenAI fine-tuning API for Wally.

In [57]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load the environment variables
load_dotenv()

# Create instance of OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [58]:
# Data setup
data_folder = "../data/processed/empathetic-conversational-model"
training_file_name = os.path.join(data_folder, "train.jsonl")
# validation_file_name = os.path.join(data_folder, "validation.jsonl")

## Upload files
Use `client.files.create()` method from OpenAI Files API to upload training file (for now only training, no validation) to OpenAI API. Afterwards, store returned File object ID for reference.

In [59]:
training_file = client.files.create(
    file=open(training_file_name, "rb"),
    purpose="fine-tune",
)

training_file_id = training_file.id
print(f"Training file ID: {training_file_id}")

Training file ID: file-DzJpfu3f5YCy87TKV9dyeS


## Create fine-tuning job
Use the `client.fine_tuning.jobs.create()` method to create a fine-tuning job

In [60]:
job = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-4o-mini-2024-07-18",
    suffix="wally",
)

job_id = job.id

print(f"Job ID: {job_id}")
print(f"Job status: {job.status}")

Job ID: ftjob-W0ZvpvAJew1BHtngvVfUFZnd
Job status: validating_files


## Check Job Status
Check status using `client.fine_tuning.jobs.retrieve() ` method, which takes in job ID.

In [71]:
retrieve_response = client.fine_tuning.jobs.retrieve(job_id)

print(f"Job ID: {retrieve_response.id}")
print(f"Job status: {retrieve_response.status}")
print(f"Model: {retrieve_response.model}")
print(f"Trained Tokens: {retrieve_response.trained_tokens}")

Job ID: ftjob-W0ZvpvAJew1BHtngvVfUFZnd
Job status: succeeded
Model: gpt-4o-mini-2024-07-18
Trained Tokens: 23188


List events of the job using the `client.fine_tuning.jobs.list_events()` method. Returns a list of events associated with the job.

In [69]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.15
Step 86/100: training loss=1.26
Step 87/100: training loss=1.35
Step 88/100: training loss=0.01
Step 89/100: training loss=0.64
Step 90/100: training loss=0.00
Step 91/100: training loss=0.19
Step 92/100: training loss=0.93
Step 93/100: training loss=0.22
Step 94/100: training loss=0.27
Step 95/100: training loss=1.63
Step 96/100: training loss=0.38
Step 97/100: training loss=1.16
Step 98/100: training loss=0.15
Step 99/100: training loss=0.52
Step 100/100: training loss=1.21
Checkpoint created at step 50
Checkpoint created at step 75
New fine-tuned model created
The job has successfully completed
